<a href="https://colab.research.google.com/github/RiccardoRubini93/2D_NS_inc/blob/master/Test01_load_into.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd 
from google.cloud import bigquery
import numpy as np
from google.api_core import page_iterator
from google.cloud import storage
import os
#import datetime
from datetime import datetime, date, timedelta
from datetime import timezone
import pandas_gbq as pdbq
import re


def convert_datetime_to_string(data):
  return data.strftime('%Y-%d-%m | %H:%M:%S')

#next step is to add the extra columns

def bucket_metadata(bucket_name,project_id):
    """Prints out a bucket's metadata."""
  
    storage_client = storage.Client(project_id)
    bucket = storage_client.get_bucket(bucket_name)

    data                  = {}
    data['bucket_id']     = bucket.id
    data['bucket_name']          = bucket.name
    data['bucket_storage_class'] = bucket.storage_class
    data['bucket_location']      = bucket.location
    data['bucket_location_type'] = bucket.location_type
    data['bucket_created'] = str(bucket.time_created)

    return data

def file_path_policy_respected_check(file_path):
    #function that checks whether the path policy of a certain bucket is respected or not
    #each path should be in the following format 
    path_chuncks = file_path.split('/')[2:-1]
    #print(path_chuncks)
    if len(path_chuncks) == 5 : policy_respected = True
    else : policy_respected = False

    return policy_respected

def _item_to_value(iterator, item):
    return item

def list_directories(bucket_name, prefix):

    if prefix and not prefix.endswith('/'):
        prefix += '/'

    extra_params = {
        "projection": "noAcl",
        "prefix": prefix,
        "delimiter": '/'
    }

    gcs = storage.Client()

    path = "/b/" + bucket_name + "/o"

    iterator = page_iterator.HTTPIterator(
        client=gcs,
        api_request=gcs._connection.api_request,
        path=path,
        items_key='prefixes',
        item_to_value=_item_to_value,
        extra_params=extra_params,
    )

    return [x for x in iterator]

#definitions of helpers functions

def file_path_pipeline_code_distinct_versions_check(input_path):
  if (input_path [-1])!='/':
    input_path+='/'
    
  bucket = input_path.split('/')[2]
  #print(bucket)
  prefix  = input_path.split(bucket)[1][1:-4]
  #print(prefix)

  #build a list containing path and corresponding sizes

  versions_sizes = []

  #print(list_directories(bucket,prefix))

  for directory in list_directories(bucket,prefix):

    path = "gs://"+ bucket + "/" +  directory 
    #print(path)
    size = os.popen("gsutil du -s " + str(path)).read().split()
    #print("Size : " + str(size))

    versions_sizes.append(size)
  return versions_sizes

def get_file_number_in_landing_zone_path(input_path):

    project_id = 'advanced-analytics-278408'
    client = storage.Client(project_id)

    bucket = input_path.split('/')[2]
    prefix  = input_path.split(bucket)[1][1:-1]

    count = 0

    for blob in client.list_blobs(bucket,prefix=prefix):
      count +=1

    #print("Landing zone path : " + str(input_path) )
    print("File number : " + str(count))

    return count

def get_file_size_in_landing_zone_path(input_path):

    bucket = input_path.split('/')[2]
    #print(bucket)
    prefix  = input_path.split(bucket)[1][1:-4]
    #print(prefix)

    # corresponding sizes

    path = "gs://"+ bucket + "/" +  prefix 
    #print(path)
    size = os.popen("gsutil du -s " + str(path)).read().split()[0]
    print("Size : " + str(size))
  
    return round(int(size) / 1024 ** 3, 2)

def get_oldest_and_newest_file(input_path,project_id):

  client = storage.Client(project_id)

  bucket_name = input_path.split('/')[2]
  prefix =      input_path.split(bucket_name)[1][1:-1]

  #generate a list of blobs

  blobs = client.list_blobs(bucket_name,prefix=prefix)

  blobs = [(blob, blob.updated) for blob in client.list_blobs(bucket_name,prefix=prefix)]
  #print(list(client.list_blobs(bucket_name,prefix = prefix)))
  
  
  #sort blobs by update date
  if len(blobs)>0:
    newest  = sorted(blobs, key=lambda tup: tup[1])[-1] 
    oldest  = sorted(blobs, key=lambda tup: tup[1])[0]
  else:
    newest='No files found'
    oldest='No files found'

  return oldest,newest

def is_file_policy_respected(input_path,project_id,bucket,prefix):

  client = storage.Client(project_id)

  name_is_conformal = True

  print("Warning message: Check that the policy of the file is respected")
  #print("Bucket : " + str(bucket))
  #print("Prefix : " + str(prefix))

  for blob in client.list_blobs(bucket,prefix=prefix):

    if len((blob.name.split('/')[-1]).split('__')) != 4 : 
      print("Warning message: Found not conformal file. The file that does not respect the policy is: " + blob.name)
      #print(blob.name)
      name_is_conformal = False
      break
  
  return name_is_conformal

def extract_file_info(input_path,project_id,bucket):

  client = storage.Client(project_id)

  #divide la stringa input_path per estrarre il bucket_name e il prefisso per filtrare i BLOB.
  bucket_name = input_path.split('/')[2]
  #print(bucket_name)
  prefix =      input_path.split(bucket)[1][1:-1]
  #print(prefix)

  #print(blob.name.split('/')[2])

  # # Crea un dizionario vuoto richiamato e un contatore cnt inizializzato a zero.
  out = {}
  cnt = 0

  if len(list(client.list_blobs(bucket,prefix=prefix)))>0:

    for blob in client.list_blobs(bucket,prefix=prefix):
      #print(blob.name)

      try:
        out['file_name_pipeline_code_info'] = blob.name.split('/')[2]
      except: out['file_name_pipeline_code_info'] = 'Missing File'

      try:
        out['file_name_pipeline_name_info'] = blob.name.split('/')[1]
      except: out['file_name_pipeline_name_info'] = 'Missing File'

      try: 
        out['file_name_pipeline_windowframe_info'] = blob.name.split('/')[5].split('__')[2]
      except: out['file_name_pipeline_windowframe_info'] = 'Missing File'

      if cnt > 0 : break

      cnt = +1

    
  else:
    out['file_name_pipeline_code_info'] = 'Missing File'
    out['file_name_pipeline_name_info'] = 'Missing File'
    out['file_name_pipeline_windowframe_info'] = 'Missing File'

  return out

#generate full list of days for the period covered 

#from datetime import datetime, date, timedelta

def generate_list(period_covered,load_time_frame):

  year_s = int(period_covered[0][0:4])
  year_e = int(period_covered[1][0:4])

  month_s = int(period_covered[0][4:6])
  month_e = int(period_covered[1][4:6])

  if load_time_frame == 'dd':

    day_s = int(period_covered[0][6:8])
    day_e = int(period_covered[1][6:8])
    freq = 'd'

  elif load_time_frame == 'mm':

    day_s =  1
    day_e =  1
    freq = 'm'

  start = date(year_s, month_s, day_s)  
  end   = date(year_e, month_e, day_e) 
  
  time_list = []

  if load_time_frame == 'mm':
    
    for item in pd.date_range(start,end,freq=freq).strftime('%Y%m%d').to_list():

      time_list.append(item[0:6])
  
  elif load_time_frame == 'dd':

    time_list = pd.date_range(start,end,freq=freq).strftime('%Y%m%d').to_list() 

  
  return time_list

# #select the oldest and newest file -> from filename

def get_file_name_pipeline_history_min_and_max(project_id, file_path):

  client = storage.Client(project_id)

  bucket_name = file_path.split('/')[2]
  prefix = file_path.split(bucket_name)[1][1:-1]
  
  blobs =[]

  pattern = re.compile(r'\d\d\d\d')
  
  for blob in client.list_blobs(bucket_name, prefix=prefix):
    if '90_sprinklr' in blob.name:
      if bool(pattern.match(blob.name.split('/')[4])):
        timestamp= None
        filename = blob.name.split('/')[-1]
        if '__' in filename:
          timestamp = filename.split('__')[-1].replace('.json','')
          blobs.append([timestamp])
          if '_' in filename:
            timestamp = filename.split('_')[-1].replace('.json','').replace('engagement','')
            blobs.append([timestamp])
    else:
      filename = blob.name.split('/')[-1]
    if '__crm_master_ID__' in filename and filename.endswith('.csv'):
      timestamp=filename.split('__')[-2]
      blobs.append([timestamp])
    elif '__mm__' in filename or '__dd__' in filename and (filename.endswith('.csv') or filename.endswith('.tsv') or filename.endswith('.json')):
          if filename.endswith('.csv'):
            timestamp = filename.split('__')[-1].replace('.csv', '')
            blobs.append([timestamp])
          elif filename.endswith('.tsv'):
              timestamp = filename.split('__')[-1].replace('.tsv', '')
              blobs.append([timestamp])
          else:
              timestamp = filename.split('__')[-1].replace('.json', '')
              blobs.append([timestamp])
    elif '__' in filename and (filename.endswith('.csv') or filename.endswith('.parquet')):
          if filename.endswith('.csv'):
            timestamp = filename.split('__')[-2]
            blobs.append([timestamp])
          else:
              timestamp = filename.split('__')[-2].replace('-','')
              blobs.append([timestamp])
    else:
      if '__' in filename:
        timestamp = filename.split('__')[-1]
        blobs.append([timestamp])

  if len(blobs) > 0:
    newest = sorted(blobs, key=lambda tup: tup[0])[-1]
    oldest = sorted(blobs, key=lambda tup: tup[0])[0]
  else:
    newest = 'No files found'
    oldest = 'No files found'

  return oldest, newest, blobs
  

def check_missing_days(list1,list2):
  #function that checks if all days are present 
  #list1 contains the set that needs to be checked
  #list2 contains the whole list

  missings = []

  for item in list2:

    if item in list1:
      pass
    else:
      missings.append(item)

  return missings

def get_warning(path, freq_agg):
  today = datetime.now(timezone.utc)

  bucket_name = path.split('/')[2]
  prefix  = path.split(bucket_name)[1][1:-1]

  storage_client = storage.Client()
  bucket = storage_client.bucket(bucket_name)

  l = []
  l2 = []

  for directory in list_directories(bucket_name,prefix):
    #print(directory)
    l.append(directory)

  blob_last_directory=max(l)
  #print(blob_last_directory)

  files = bucket.list_blobs()    
  fileList = [file.name for file in files if blob_last_directory in file.name]

  for file in fileList:
    blob = bucket.get_blob(file)
    l2.append(blob.updated)
    #print(blob.updated)
  
  blob_last_mod=max(l2)
  #print(blob_last_mod)

  if freq_agg=='dd':
    if (today - blob_last_mod).days < 1:
      return 'green'
    elif (today - blob_last_mod).days >= 1 and (today - blob_last_mod).days <= 3:
      return 'yellow'
    else:
      return 'red'
      
  if freq_agg=='ww':
    if (today - blob_last_mod).days < 7:
      return 'green'
    elif (today - blob_last_mod).days >= 7 and (today - blob_last_mod).days <= 10:
      return 'yellow'
    else:
      return 'red'
  else:
    if (today - blob_last_mod).days < 30:
      return 'green'
    elif (today - blob_last_mod).days >= 30 and (today - blob_last_mod).days <= 45:
      return 'yellow'
    else:
      return 'red'
  
def populate_9898(df,df_9898,i,columns_list,project_id):

  #print the filepath analised
  print("Computing row : " + str(i) + " of " + str(df.shape[0]))
  #print(df.loc[i,'landing_zone_path'])
  print("landing zone path: " + df.loc[i,'landing_zone_path'])

  #fill the date

  df_9898.loc[i,'analysis_date'] = datetime.now().strftime('%Y-%m-%d')
  df_9898.loc[i,'analysis_date_time'] = datetime.now()

  #add columns from the 9891
  for col in columns_list:

    df_9898.loc[i,col] = df.loc[i,col]

  bucket = df.loc[i,'landing_zone_path'].split('/')[2]
  bucket_meta = bucket_metadata(bucket,project_id)
  
  #add bucket metadata
  for key in bucket_meta:
    df_9898.loc[i,key] = bucket_meta[key] 

  #check that the policy and path info
  df_9898.loc[i,'file_path_policy_respected'] = file_path_policy_respected_check(df_9898.loc[i,'landing_zone_path'])
  df_9898.loc[i,'file_path_source_system'] = df_9898.loc[i,'landing_zone_path'].split('/')[5]
  df_9898.loc[i,'file_path_pipeline_code'] = df_9898.loc[i,'landing_zone_path'].split('/')[4]


  #extract bucket and prefix for a given landing_zone_path
  bucket = df_9898.loc[i,'landing_zone_path'].split('/')[2]
  prefix  = df_9898.loc[i,'landing_zone_path'].split(bucket)[1][1:-4]
  #count the number of v1 v2 v3 
  #print(bucket)
  #print(prefix)
  print(f"List directories : {list_directories(bucket,prefix)}")

  df_9898.loc[i,'file_path_pipeline_code_distinct_versions'] = len(list_directories(bucket,prefix))

  #compute the size of each versioning of each folder
  
  versions = file_path_pipeline_code_distinct_versions_check(df_9898.loc[i,'landing_zone_path'])

  df_9898.loc[i,'file_path_pipeline_code_distinct_versions_sizes'] = str(versions)
  df_9898.loc[i,'file_path_pipeline_code_checked_version'] = str(versions[-1][1])

  #compute the number of files in the landing zone path

  df_9898.loc[i,'file_number_in_landing_zone_path'] = get_file_number_in_landing_zone_path(df_9898.loc[i,'landing_zone_path'],)
 
  df_9898.loc[i,'file_size_in_landing_zone_path'] = get_file_size_in_landing_zone_path(df_9898.loc[i,'landing_zone_path'])

  #get oldest and most recent file in folder

  new = get_oldest_and_newest_file(df_9898.loc[i,'landing_zone_path'],project_id)[0][1]
  old = get_oldest_and_newest_file(df_9898.loc[i,'landing_zone_path'],project_id)[1][1]

  # df_9898.loc[i,'file_oldest_creation_datetime'] = old
  # df_9898.loc[i,'file_newest_creation_datetime'] = new

  df_9898.loc[i,'file_oldest_creation_datetime'] = new
  df_9898.loc[i,'file_newest_creation_datetime'] = old

  #check policy of the file
  df_9898.loc[i,'file_name_policy_respected']  = is_file_policy_respected(df_9898.loc[i,'landing_zone_path'],project_id,bucket,prefix)

  #extract file infos

  files_info = extract_file_info(df_9898.loc[i,'landing_zone_path'],project_id,bucket)

  df_9898.loc[i,'file_name_pipeline_code_info']        =  files_info['file_name_pipeline_code_info']
  df_9898.loc[i,'file_name_pipeline_name_info']        =  files_info['file_name_pipeline_name_info']
  df_9898.loc[i,'file_name_pipeline_windowframe_info'] =  files_info['file_name_pipeline_windowframe_info']

  #get first and latest date from filename

  old,new,list_day = get_file_name_pipeline_history_min_and_max(project_id,df_9898.loc[i,'landing_zone_path'])
  period_covered = [old[0],new[0]]

  df_9898.loc[i,'file_name_pipeline_history_min_info'] = old[0]
  df_9898.loc[i,'file_name_pipeline_history_max_info'] = new[0]
  df_9898.loc[i,'files_period_covered'] = str(period_covered)

  #get the complete list of days
  try : 
    complete_list_days = generate_list(period_covered,df_9898.loc[i,'file_name_pipeline_windowframe_info'])
    missings = check_missing_days(np.asarray(list_day)[:,0],complete_list_days)

    if len(missings) == 0 : df_9898.loc[i,'files_period_missing_between_min_max'] = 'No missing days'
    else: df_9898.loc[i,'files_period_missing_between_min_max'] = str(missings)
  except:
    print("Missing Files, step skipped")
  
  #get_warning_status
  try: df_9898.loc[i,'warning_flag_status'] = get_warning(df_9898.loc[i,'landing_zone_path'],df_9898.loc[i,'file_name_pipeline_windowframe_info'])
  except : df_9898.loc[i,'warning_flag_status'] = '' 

  return df_9898

def write_to_table(client,df_9898,bq_dataset_id,table_name):

    dataset_ref = client.dataset(bq_dataset_id)
    job_config = bigquery.LoadJobConfig()
    job_config.autodetect = False
    job_config.write_disposition = "WRITE_APPEND"
    load_job = client.load_table_from_dataframe(df_9898, dataset_ref.table(table_name), job_config=job_config, location="EU")    # API request
    print("Starting job {}".format(load_job))

    return True


def load_into_9898():

    project_id = 'advanced-analytics-278408'
    client = bigquery.Client(project=project_id)

    query = '''
        SELECT
        *
        FROM `advanced-analytics-278408.data__1st_layer.9891__monitoring_information_and_params`
        WHERE is_to_be_monitored = true
        and landing_zone = 'GCS';
        '''

    df = client.query(query).to_dataframe()
        #df.columns

    query_2 = '''
        SELECT
        *
        FROM `advanced-analytics-278408.user__rubini_riccardo.9898__pipelines_target_landing_zone_metadata_ori`;
        '''

    df_9898 = client.query(query_2).to_dataframe()

        #fill the first N column of df_9898 with the first N columns of 9891__monitoring_information_and_params
        #define a list of columns to fill 

    columns_list = [
                'project_id', 'dataset_id', 'table_id', 'complete_table_id',
                'table_code', 'is_to_be_monitored', 'current_development_status',
                'current_development_status_id', 'source_system_id',
                'source_system_name', 'source_system_ownership_type', 'extraction_tool',
                'extraction_script', 'extraction_is_scheduled',
                'extraction_schedule_pattern', 'landing_zone', 'landing_zone_path',
                'landing_zone_form', 'loading_tool', 'load_script', 'load_is_scheduled',
                'load_schedule_pattern', 'load_form'
        ]

        #loop over the columns to fill 

    #drop and recreate table 
    table_id = 'advanced-analytics-278408.user__rubini_riccardo.9898__pipelines_target_landing_zone_metadata_test'
    client.delete_table(table_id, not_found_ok=True)  # Make an API request.
    print("Deleted table '{}'.".format(table_id))


        #for i in range(20):
    for i in range(df.shape[0]):

            
                df_9898_ = populate_9898(df,df_9898,i,columns_list,project_id)
                pd.concat([df_9898, df_9898_])
                print(df_9898_.iloc[[i]])
                
                
                try : 
                  write_to_table(client,df_9898_.iloc[[i]],bq_dataset_id='user__rubini_riccardo',table_name='9898__pipelines_target_landing_zone_metadata_test')
                  print(f'Writing Row {i}')

                except : print(f'Failed to write Row {i}')

    df_final = df_9898_.astype(str)

    pdbq.to_gbq(df_final, 
                'user__rubini_riccardo.9898__pipelines_target_landing_zone_metadata_test_backup', 
                    project_id, if_exists='append') 